## Librerías 

In [2]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd
from sklearn.utils import class_weight
from sklearn import tree
import a_funciones as fn

import cv2 ### para leer imagenes jpeg
### pip install opencv-python

from matplotlib import pyplot as plt #

2024-11-05 18:23:34.621178: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# cargar bases_procesadas con función
x_train, y_train, x_test, y_test, x_val, y_val = fn.imag_array()

100%|██████████| 227/227 [00:01<00:00, 115.04it/s]


In [4]:
x_train.max()

255

In [5]:
x_test.max()

255

In [6]:
# Se procede a escalar sobre su máximo valor 
x_train = x_train.astype('float32') #convertir a float para escalar
x_test = x_test.astype('float32') #convertir a float para escalar
x_train /= 255 #sobre el máximo valor de la representación numérica de las imágenes
x_test /= 255

# dimensionalidad
x_train.shape
x_test.shape
# Se obtiene cantidad de imágenes 
# Representación de arrays de 224 x 224 (pixeles)
#Canales = 3

(336, 224, 224, 3)

In [7]:
np.product(x_train[1].shape) # número de columnas

150528

In [8]:
np.unique(y_train, return_counts=True) #conteo de las clases

(array([0, 1]), array([1569,  803]))

In [9]:
np.unique(y_test, return_counts=True) #conteo de las clases

(array([0, 1]), array([208, 128]))

In [10]:
# Array de 1 dimensión
x_train2 = x_train.reshape(2372,150528)
x_test2 = x_test.reshape(336, 150528)
print(x_train2.shape)
x_test2.shape

(2372, 150528)


(336, 150528)

### Selección de métricas 

Recall y AUC (En el informe se justificó la razón)

### 1. Modelo Random Forest

In [20]:
modelo_1=RandomForestClassifier() #bosques aleatorios
modelo_1.fit(x_train2, y_train.ravel()) #entrenar modelo

train_pred=modelo_1.predict(x_train2)
Recall=metrics.recall_score(y_train, train_pred, average=None)
print(f"La métrica Recall para entrenamiento y clase 0: {Recall[0]}") #Recall Train clase 0 (no cáncer)
print(f"La métrica Recall para entrenamiento y clase 1: {Recall[1]}") #Recall Train clase 1 (cáncer)

La métrica Recall para entrenamiento y clase 0: 1.0
La métrica Recall para entrenamiento y clase 1: 1.0


In [21]:
print(f"La métrica AUC para entrenamiento dio: {metrics.roc_auc_score(y_train, train_pred)}") #AUC de train

La métrica AUC para entrenamiento dio: 1.0


In [22]:
test_pred=modelo_1.predict(x_test2)
Recall=metrics.recall_score(y_test, test_pred, average=None)
print(f"La métrica Recall para evaluación y clase 0: {Recall[0]}") #Recall Test clase 0 ( no cáncer)
print(f"La métrica Recall para evaluación y clase 1: {Recall[1]}") #Recall Test clase 1 (cáncer)

La métrica Recall para evaluación y clase 0: 0.9663461538461539
La métrica Recall para evaluación y clase 1: 0.09375


In [23]:
print(f"La métrica AUC para evaluación dio: {metrics.roc_auc_score(y_test, test_pred)}") #AUC de test

La métrica AUC para evaluación dio: 0.530048076923077


### 2. Modelo de Redes Neuronales Estándar

In [ ]:
y_train = y_train.ravel() #convierte y_train en un arreglo unidimensional


# Definir el modelo de red neuronal con regularización Dropout
fc_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=x_train.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),  # Dropout para reducir el sobreajuste; 'apaga' el 30% de neuronas
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

c:\Users\delva\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
#Configurar el optimizador
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) #busca minimizar la función de pérdida del modelo


In [12]:
# Calcular los pesos de cada clase para mejorar el aprendizaje de las clases desbalanceadas
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

#Compilar el modelo
fc_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['Recall', 'AUC'])


In [13]:
#Entrenamiento del modelo
fc_model.fit(
    x_train, y_train,
    batch_size=32,  #tamaño de lote reducido
    epochs=20,  #mas iteraciones para una mejor convergencia
    validation_data=(x_test, y_test),
    class_weight=class_weights_dict
)

# Evaluar el modelo
test_loss, test_recall, test_auc = fc_model.evaluate(x_test, y_test, verbose=2)
print("Test recall:", test_recall)
print("Test AUC:", test_auc)
print("Test loss:", test_loss)


Epoch 1/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 76ms/step - AUC: 0.5110 - Recall: 0.4653 - loss: 0.8581 - val_AUC: 0.5198 - val_Recall: 0.0859 - val_loss: 0.6776
Epoch 2/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - AUC: 0.4899 - Recall: 0.4487 - loss: 0.7191 - val_AUC: 0.5231 - val_Recall: 0.4688 - val_loss: 0.6916
Epoch 3/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - AUC: 0.4942 - Recall: 0.6712 - loss: 0.6986 - val_AUC: 0.5087 - val_Recall: 1.0000 - val_loss: 0.6949
Epoch 4/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - AUC: 0.4869 - Recall: 0.7689 - loss: 0.6979 - val_AUC: 0.5101 - val_Recall: 0.9531 - val_loss: 0.6963
Epoch 5/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 5s 50ms/step - AUC: 0.5035 - Recall: 0.7750 - loss: 0.6902 - val_AUC: 0.4965 - val_Recall: 0.9922 - val_loss: 0.6940
Epoch 6/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - AUC: 0.5123 - Recall: 0.9445 - loss: 0.6956 - val_AUC: 0.4973 - val_Recall: 1.0000 - val_loss: 0.6942
Epoch 7/20
75/75 ━━━━━━━━━━━━━━━━━━━━ 4s 53ms/step - AUC: 0.5042 - Re